<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/0711.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1.

In [ ]:
# norm corr 可以增加
if peak < 0.2 or ratio < min_corr_ratio:
    lag = 0

## 2.

**方法:**

1. 將 reference 和 target 都切成特徵序列：
Reference：R = [r_0, r_1, ..., r_n]

Target：T = [t_0, t_1, ..., t_m]

2. 針對每個可能的 lag（移動距離），對齊 R 和 T[lag : lag+n]
計算這兩段對應 patch 間的 cosine similarity

平均相似度作為該 lag 的分數

**注意:**

1. 要確保 target 長度要夠大，讓你可以從頭到尾滑動比較。如果 target 比 ref 短，就沒地方滑。

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

# -------- CNN 特徵擷取器 --------
class CNNEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=5, padding=2),  # 1D卷積層:輸入1通道，輸出16通道，卷積核長度5，padding=2維持長度不變
            nn.ReLU(),                                   # ReLU非線性激活函數
            nn.Conv1d(16, 8, kernel_size=3, padding=1), # 第二層卷積: 16通道輸入，8通道輸出，核長3，padding=1維持長度
            nn.AdaptiveAvgPool1d(1)                      # 自適應平均池化到長度1，將每個channel壓縮成單一數值（全域平均池化）
        )

    def forward(self, x):
        return self.conv(x).squeeze(-1)  # 執行卷積與池化，並移除最後維度1，使輸出形狀變成 (batch_size, 8)

# -------- 特徵抽取 (切成滑窗) --------
def extract_features(signal, window_size, cnn):
    cnn.eval()   # 設定 CNN 為評估模式，關閉 dropout 等訓練時行為
    features = []
    for i in range(len(signal) - window_size + 1):  # 對訊號用滑動視窗切片
        segment = signal[i:i+window_size]           # 取第 i 個 window_size 長度的片段
        with torch.no_grad():                        # 不計算梯度，節省記憶體與速度加快
            x = torch.tensor(segment, dtype=torch.float32).view(1, 1, -1)  # 將片段轉為 Tensor，格式(batch=1, channel=1, length)
            feat = cnn(x).numpy().flatten()         # 通過 CNN 抽特徵，轉成 numpy 陣列，展平成1維向量
        features.append(feat)                        # 將特徵向量加入清單
    return np.array(features)                        # 回傳所有滑窗特徵，形狀為 (num_windows, feature_dim)

# -------- 主函式：全段對齊 --------
def align_full_sequence(reference, target, cnn, window_size=50, max_lag=30):
    ref_feat = extract_features(reference, window_size, cnn)  # 取得 reference 訊號所有滑窗特徵
    tar_feat = extract_features(target, window_size, cnn)     # 取得 target 訊號所有滑窗特徵

    score_per_lag = []  # 用來存每個 lag 的相似度分數
    for lag in range(-max_lag, max_lag+1):  # 對 lag 從 -max_lag 到 +max_lag 做迴圈
        if lag < 0:
            ref_aligned = ref_feat[:lag]       # 當 lag 為負，reference 往左滑，所以取前面部分（切短尾巴）
            tar_aligned = tar_feat[-lag:]      # target 往右滑，取後面部分（切短頭）
        elif lag > 0:
            ref_aligned = ref_feat[lag:]       # lag 正，reference 往右滑，切掉前面部分
            tar_aligned = tar_feat[:-lag]      # target 往左滑，切掉尾巴
        else:
            ref_aligned = ref_feat              # lag = 0，對齊整段
            tar_aligned = tar_feat

        if len(ref_aligned) == 0 or len(tar_aligned) == 0:  # 如果切出來的特徵長度為0，跳過
            score_per_lag.append(0)
            continue

        sim = cosine_similarity(ref_aligned, tar_aligned)  # 計算兩組特徵間的 cosine 相似度矩陣，形狀為 (len(ref_aligned), len(tar_aligned))
        avg_sim = np.mean(np.diag(sim))  # 取對角線的值代表一對一對齊位置的相似度，求平均得到該 lag 的平均相似度
        score_per_lag.append(avg_sim)    # 將該 lag 分數加入列表

    best_lag = np.argmax(score_per_lag) - max_lag  # 找出最大分數所在的索引，轉換成 lag 值（因為索引是從 0 到 2*max_lag）
    return best_lag, score_per_lag

# -------- 測試用例 --------
np.random.seed(42)  # 固定亂數種子，方便重現
length = 200
ref_signal = np.sin(np.linspace(0, 4 * np.pi, length)) + np.random.normal(0, 0.1, length)  # 產生有噪聲的正弦波做為 reference
target_signal = np.concatenate([np.zeros(10), ref_signal[:-10]]) + np.random.normal(0, 0.1, length)  # target 往右平移10點，加噪聲

cnn = CNNEncoder()  # 建立 CNN 特徵擷取模型
lag, scores = align_full_sequence(ref_signal, target_signal, cnn, window_size=40, max_lag=20)  # 呼叫對齊函式，估算最佳 lag

# -------- 可視化 --------
plt.plot(range(-20, 21), scores)  # 繪製不同 lag 的平均相似度曲線
plt.axvline(x=lag, color='red', linestyle='--', label=f'Best Lag = {lag}')  # 標出最佳 lag 位置
plt.title('CNN-based Full-sequence Alignment')
plt.xlabel('Lag')
plt.ylabel('Average Cosine Similarity')
plt.legend()
plt.grid()
plt.show()



* max_lag 是你允許搜尋的最大位移範圍（lag），代表函式會嘗試從 -max_lag 到 +max_lag 的位移去找最佳對齊。
* max_lag 越大，計算越慢（因為需要嘗試更多的位移），也容易被錯誤峰值誤導



In [ ]:
## 實務上 可 限制不超過訊號長度十分之一，避免搜尋太大又浪費。
max_lag = min(50, len(signal)//10)